In [1]:
import tkinter
import cv2
import numpy
from matplotlib import pyplot as PLT
import scipy.io
from sklearn.preprocessing import normalize
import os
from tkinter import filedialog
import tkinter as tk
from tkinter import Label, Button
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

In [2]:
def getUltraSoundReadingFromImage(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    array_shape = image.shape
    tiles = []

    tiles = [[0 for _ in range(array_shape[0])] for _ in range(array_shape[1])]

    for x in range(0, array_shape[1]):
        for y in range(0, array_shape[0]):
            tiles[x][y] = image[y][x]

    # timeFrame: Y value of the frirst occurance of the line color (158, 210, 161) from above
    timeframe_ultrasound_mappping = {}

    maxYPixelGreenColor = 200

    for x in range(0, array_shape[1]):
        for y in range(0, array_shape[0]):
            rgb = tiles[x][y]
            if (rgb[1] >= maxYPixelGreenColor):
                timeframe_ultrasound_mappping.update({x : y})
    return timeframe_ultrasound_mappping

def changeYValuesToMatchNormalmatrix(mapping, image):
    maxY = image.shape[0]

    for k,v in mapping.items():
        mapping[k] = maxY - v


def calculateAverageEcgData(lastTimeFrameOfEcg, ultraSoundDataRatio):
    averagedEcgData = []
    i = 1
    average = 0
    for ecgPoint in lastTimeFrameOfEcg:
        if (i == ultraSoundDataRatio):
            averagedEcgData.append( average / ultraSoundDataRatio )
            i = 1
            average = 0
        else:
            average = average + ecgPoint
            i = i + 1
    return averagedEcgData

def getFirstValuesOfDic(dictionary, elementsCount):
    values = []
    for key in sorted(dictionary)[:elementsCount]:
        values.append(dictionary[key])
    return values

def normalizeValuesTo100(values):
    values = normalize([values], norm="max")
    return [int(x * 100) for x in values[0]]



def getEcgPeakTimes(ecgData, threshold):
    ecgPeakTimes = []
    for i, ecgPoint in enumerate(ecgData):
        if (ecgPoint >= threshold):
            ecgPeakTimes.append ((i, ecgPoint))
    newEcgPeakTimes = []
    i = 0
    while (i < len(ecgPeakTimes)):
        ecgPeakTime = ecgPeakTimes[i][0]
        ecgPeakValue = ecgPeakTimes[i][1]

        j = 1
        while( i + j < len(ecgPeakTimes)):
            if (ecgPeakTimes[ i + j ][0]  - ecgPeakTime < 30 ):
                ecgPeakTime = (ecgPeakTime + ecgPeakTimes[ i + j ][0]) / 2
                ecgPeakValue = (ecgPeakValue + ecgPeakTimes[ i + j ][1]) / 2
                j = j + 1 
            else:
                break
        i = i + j
        newEcgPeakTimes.append((ecgPeakTime, ecgPeakValue))
    return newEcgPeakTimes


def getUltrasoundDipTimes(timeframeUltrasoundData):
    ultrasoundDipTimes = []
    i = 0
    peakFlag = False
    while ( i < len(timeframeUltrasoundData) ):
        if ( timeframeUltrasoundData[i] >= 80 and not peakFlag ):
            peakFlag = True
            j = 0
            while( i - j > 0):
                if ( timeframeUltrasoundData[i - j] >= 20 and timeframeUltrasoundData[i - j] <= 30):
                    ultrasoundDipTimes.append((i - j, timeframeUltrasoundData[i - j]))
                    break
                j = j + 1

        elif (timeframeUltrasoundData[i] < 80):
            peakFlag = False
        i = i + 1 
    return ultrasoundDipTimes



def calculateDeltaTinMs(ecgPeakTimes, ultrasoundDipTimes, ultraSoundDataRatio):
    dt = 0
    for i in range(3):
        dt = dt + (ecgPeakTimes[i][0] - ultrasoundDipTimes[i][0])
    return int ( (dt / 3) * ultraSoundDataRatio )



In [3]:
def calculateDeltaT(ultraSoundImagePath, ecgFilePath):
    # let's suppose the time frame is 4000 ms
    timeFrameInMs = 4000

    ultrasoundImage = cv2.imread(ultraSoundImagePath)
    print("image shape:", ultrasoundImage.shape)

    timeframe_ultrasound_mappping = getUltraSoundReadingFromImage(ultrasoundImage)
    changeYValuesToMatchNormalmatrix(timeframe_ultrasound_mappping, ultrasoundImage)

    ultraSoundDataRatio = int(timeFrameInMs / ultrasoundImage.shape[1])
    print("every x from the ultrasound expresses ", ultraSoundDataRatio, "ms worth of data from the ECG")

    ecgData = scipy.io.loadmat(ecgFilePath)['data']
    ecgData = [ecgPoint[0] for ecgPoint in ecgData]
    lastTimeFrameOfEcg = ecgData[-timeFrameInMs:]
    averagedEcgData = calculateAverageEcgData(lastTimeFrameOfEcg, ultraSoundDataRatio)

    minimumDataLength = min(len(timeframe_ultrasound_mappping), len(averagedEcgData))
    
    timeframeUltrasoundData = getFirstValuesOfDic(timeframe_ultrasound_mappping, minimumDataLength)
    timeframeUltrasoundData = normalizeValuesTo100(timeframeUltrasoundData)
    ultrasoundDipTimes = getUltrasoundDipTimes(timeframeUltrasoundData)
    
    averagedEcgData = averagedEcgData[0:minimumDataLength]
    averagedEcgData = normalizeValuesTo100(averagedEcgData)
    ecgPeakTimes = getEcgPeakTimes(averagedEcgData, 80)

    result = calculateDeltaTinMs(ecgPeakTimes, ultrasoundDipTimes, ultraSoundDataRatio)
    print(result, 'ms')
    return result

In [4]:
class UserInterface:
    photo = None

    def __init__(self, window):
        self.window = window
        self.window.title("PULS WAVE Velocity")
        self.window.geometry("1200x1000")

       

        # Assuming 'kreislauf.png' is your actual image file name
        image_filename = 'userfaceImage.png'
        
        # Create the complete path
        image_path = os.path.join(os.getcwd(), image_filename)
        
        if not UserInterface.photo:
            UserInterface.photo = tk.PhotoImage(file=image_path)
        
        self.image_label = tk.Label(window, image=UserInterface.photo)
        self.image_label.grid(row=4, column=5, rowspan=5)


        # Age Input
        age_label = tk.Label(window, text="Enter person Age:")
        age_label.grid(row=5, column=0, sticky=tk.E)
        self.age_entry = tk.Entry(window)
        self.age_entry.grid(row=5, column=1)

        # carotid-femoral distance Input
        distance_label = tk.Label(window, text="Enter carotid-femoral distance in cm:")
        distance_label.grid(row=6, column=0, sticky=tk.E)
        self.distance_entry = tk.Entry(window)
        self.distance_entry.grid(row=6, column=1)

        # Gender Selection
        gender_label = tk.Label(window, text="Select Gender:")
        gender_label.grid(row=4, column=0, sticky=tk.E)

        self.gender_var = tk.StringVar(value="Male")
        male_button = tk.Radiobutton(window, text="Male", variable=self.gender_var, value="Male")
        female_button = tk.Radiobutton(window, text="Female", variable=self.gender_var, value="Female")

        male_button.grid(row=4, column=1, sticky=tk.W)
        female_button.grid(row=4, column=2, sticky=tk.W)

        # Buttons
        neck_image_button = tk.Button(window, text="Upload Carotid Artery Ultrasound Image in 4 sec", command=self.upload_neck_image)
        leg_image_button = tk.Button(window, text="Upload the second Artery point(ex.Femoral Artery) Ultrasound Image in 4 sec", command=self.upload_leg_image)
        ecg_button = tk.Button(window, text="Upload ECG Data (.mat file)", command=self.upload_ecg_data)
        submit_button = tk.Button(window, text="Submit", command=self.submit)

        # Use grid() to control the placement
        neck_image_button.grid(row=4, column=10, columnspan=2, pady=15)
        leg_image_button.grid(row=8, column=10, columnspan=2, pady=10)
        ecg_button.grid(row=5, column=9, columnspan=2, pady=10)
        submit_button.grid(row=20, column=40, columnspan=2, pady=10)

    def upload_neck_image(self):
        self.neck_image_path = filedialog.askopenfilename(title="Select Neck Ultrasound Image", initialdir="C:/Users/herrn/OneDrive/Desktop", filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])

    def upload_leg_image(self):
        self.leg_image_path = filedialog.askopenfilename(title="Select Leg Ultrasound Image", initialdir="C:/Users/herrn/OneDrive/Desktop", filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])

    def upload_ecg_data(self):
        self.ecg_path = filedialog.askopenfilename(title="Select ECG Data (MAT file)", initialdir="C:/Users/herrn/OneDrive/Desktop", filetypes=[("MAT files", "*.mat")])

    def submit(self):
        self.age = int(self.age_entry.get())
        self.distance = int(self.distance_entry.get())
    

        # Perform operations with the collected data
        print("Age:", self.age)
        print("carotid-femoral distance in cm:", self.distance)
        print("Neck Ultrasound Image Path:", self.neck_image_path)
        print("Leg Ultrasound Image Path:", self.leg_image_path)
        print("ECG Data Path:", self.ecg_path)
    

        
        self.window.destroy()


In [5]:
def getResultMessage(age,pwv):
    pwvString = "{:.2f}".format(pwv)
    if (age > 20 and age <40):
        if (pwv>4.2 and pwv<4.9):
            return "You are {} Years old  and you'r pwv {} m/s.\n Value is Normal.\n You are Healthy".format(age, pwvString)
        elif (pwv > 4.95):
            return "You are {} Years old and you'r pwv {} m/s.\n Value is bigger than the Normal.\n Suspected diseases : Atherosclerosis / Vascular Stiffening  ".format(age, pwvString)
        else:
           return "You are {} Years old and you'r pwv {} m/s . \n Value is smaller than the Normal.\n Suspected diseases : Vascular narrowing / Vascular Inflammatory Conditions".format(age, pwvString)
    elif (age > 40 and age <51):
        if (pwv>4.9 and pwv<5.33):
            return "You are {} Years old  and you'r pwv {} m/s.\n Value is Normal.\n You are Healthy".format(age, pwvString)
        elif (pwv > 5.33):
             return "You are {} Years old and you'r pwv {} m/s.\n Value is bigger than the Normal.\n Suspected diseases : Atherosclerosis / Vascular Stiffening  ".format(age, pwvString)
        else:   
            return "You are {} Years old and you'r pwv {} m/s . \n Value is smaller than the Normal.\n Suspected diseases : Vascular narrowing / Vascular Inflammatory Conditions".format(age, pwvString)
    elif (age > 50 and age <61):
        if (pwv>5.63 and pwv<6.45):
            return "You are {} Years old  and you'r pwv {} m/s.\n Value is Normal.\n You are Healthy".format(age, pwvString)
        elif (pwv > 6.45):
            return "You are {} Years old and you'r pwv {} m/s.\n Value is bigger than the Normal.\n Suspected diseases : Atherosclerosis / Vascular Stiffening  ".format(age, pwvString)
        else:
            return "You are {} Years old and you'r pwv {} m/s . \n Value is smaller than the Normal.\n Suspected diseases : Vascular narrowing / Vascular Inflammatory Conditions".format(age, pwvString)
    elif (age > 60 and age <71):
        if (pwv>6.35 and pwv<7.19):
            return "You are {} Years old  and you'r pwv {} m/s.\n Value is Normal.\n You are Healthy".format(age, pwvString)
        elif (pwv > 7.20):
            return "You are {} Years old and you'r pwv {} m/s.\n Value is bigger than the Normal.\n Suspected diseases : Atherosclerosis / Vascular Stiffening  ".format(age, pwvString)
        else:
            return "You are {} Years old and you'r pwv {} m/s . \n Value is smaller than the Normal.\n Suspected diseases : Vascular narrowing / Vascular Inflammatory Conditions".format(age, pwvString)

In [6]:
def quadratic_equation(x, a, b, c):
    return a * x**2 + b * x + c

def getAgeAndPwvGraph():
    age_values = np.array([27, 35, 45, 55, 71])  # Example age values
    pwv_values = np.array([4.74999999, 5.0, 5.2, 6.1, 7.19])  # Example corresponding PWV values
    
    fit_params, _ = curve_fit(quadratic_equation, age_values, pwv_values)
    
    fit_age_values = np.linspace(min(age_values), max(age_values), 100)
    fit_pwv_values = quadratic_equation(fit_age_values, *fit_params)
    return (fit_age_values, fit_pwv_values)

In [7]:
def calculatePwv(app):
    cwd = os.getcwd()
    neckUltrasoundImagePath = app.neck_image_path
    armUltrasoundImagePath = app.leg_image_path
    ecgFilePath = app.ecg_path
    
    neckDeltaT = calculateDeltaT(neckUltrasoundImagePath, ecgFilePath)
    armDeltaT = calculateDeltaT(armUltrasoundImagePath, ecgFilePath)
    
    neckArmDeltaT = (armDeltaT - neckDeltaT) / 1000
    
    print(app.distance)
    distance = (app.distance / 100) * 0.8
    return distance / neckArmDeltaT

In [8]:
window = tk.Tk()
app = UserInterface(window)
window.mainloop()

fit_age_values, fit_pwv_values = getAgeAndPwvGraph()
pwv = calculatePwv(app)
age = app.age
resultMessage = getResultMessage(age, pwv)

print(pwv)
print(resultMessage)


# Create a new window
result_window = tk.Tk()
result_window.title("User Result")



# Create a figure and axis for the graph
fig = Figure(figsize=(15, 10), dpi=100)
ax = fig.add_subplot(121)

ax.scatter(age, pwv, color='red', label='Calculated PWV ')
ax.plot(fit_age_values, fit_pwv_values, color='blue', label='PWV/Age')

ax.set_xlabel('Age')
ax.set_ylabel('PWV Value (m/s)')
ax.set_title('PWV/Age')
ax.legend()

# Embed the matplotlib graph into the Tkinter window
canvas = FigureCanvasTkAgg(fig, master=result_window)
canvas.draw()
canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)



ax = fig.add_subplot(122)
fig.subplots_adjust(top=0.85)
ax.set_axis_off()

# Set both x- and y-axis limits to [0, 10] instead of default [0, 1]
ax.axis([0, 10, 0, 10])

ax.text(0, 
        7, 
        'Age : {}'.format(age), 
        style='italic', 
        bbox={'facecolor': 'blue', 'alpha': 0.5, 'pad': 10}
       )

ax.text(0, 
        6, 
        'Your PWV : {} m/s'.format ("{:.2f}".format(pwv)), 
        style='italic', 
        bbox={'facecolor': 'blue', 'alpha': 0.5, 'pad': 10}
       )

ax.text(0, 
        3, 
        resultMessage, 
        style='italic', 
        bbox={'facecolor': 'green', 'alpha': 0.5, 'pad': 10}
       )
result_window.mainloop()

AttributeError: 'UserInterface' object has no attribute 'neck_image_path'